In [0]:
import json
import gspread
#from config import *
#from kp_scope_config import * 

def get_gspread_client_from_secret(scope='krista-ds-loc-scope',
                                   key = 'crafty_json_key'):
    
    try:
        crafty_json = dbutils.secrets.get(scope = 'krista-ds-loc-scope', key = 'crafty_json_key')
    except:
        print("Cannot get secret from scope for gspread!!")

    gc = gspread.service_account_from_dict(json.loads(crafty_json))

    print("Getting gspread client...")
    return gc


def get_gpt_secret_keys():

    """
    Get the secret open ai api keys for the current user.
    """

    try:
        current_user = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get().split("@")[0]
        scope_name = f'openai-{current_user}'
        api_user_key = dbutils.secrets.get(scope = scope_name, key = "token")
    except Exception as e:
        raise Exception(str(e))

    try:
        dbutils.secrets.list(scope = "openai")
        org_key = dbutils.secrets.get(scope = "openai", key = "token_org")
        return (current_user, api_user_key, org_key)
    except Exception as e:
        raise Exception(str(e))


def get_model_client(model_type:str = 'gpt'):
    
    if model_type not in ['gpt','gemini']:
        print(f""" The model type: {model_type} is not supported. Please choose either 'gpt' or 'gemini'""")
        return None

    if model_type == 'gpt':
        CURRENT_USER, PROJECT_API_KEY, ORGANIZATION_API_KEY =   get_gpt_secret_keys()
        client = OpenAI(
            api_key=PROJECT_API_KEY,
            organization=ORGANIZATION_API_KEY
        )

        print('Getting gpt client...')
        return client
    
    #if model_type=='gemini':

    return